# Random Walks Dashboard

Este notebook presenta un dashboard interactivo para simular distintos tipos de caminatas aleatorias en 2D y visualizarlas en 3D. Se usan NumPy para los cálculos, Plotly para la visualización y ipywidgets para los controles.

Cada simulación calcula dos estadísticas: **MDR** (desplazamiento medio) y **ML** (desplazamiento máximo). Se muestra un único par de gráficos: uno 3D (trayectoria) y uno 2D (estadística vs. paso).

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

# np.random.seed(42)  # Descomentar para reproducibilidad

In [ ]:
def brownian_motion(n_steps: int, x0: float, y0: float):
    r"""Simula un movimiento browniano en 2D.
    n_steps: número de pasos; x0, y0: posición inicial.
    Retorna listas de x e y."""
    x = [x0]
    y = [y0]
    for _ in range(n_steps):
        x.append(x[-1] + np.random.normal())
        y.append(y[-1] + np.random.normal())
    return x, y

def levy_flight(n_steps: int, alpha: float, loc: float):
    r"""Simula un Lévy Flight en 2D.
    n_steps: número de pasos; alpha: forma de la distribución Pareto; loc: longitud mínima.
    Retorna listas de x e y."""
    x, y = [0], [0]
    for _ in range(n_steps):
        step_length = np.random.pareto(alpha) + loc
        angle = np.random.uniform(0, 2 * np.pi)
        x.append(x[-1] + step_length * np.cos(angle))
        y.append(y[-1] + step_length * np.sin(angle))
    return x, y

def correlated_random_walk(n_steps: int, c: float):
    r"""Simula una caminata aleatoria correlacionada en 2D.
    n_steps: número de pasos; c: desviación estándar del cambio en el ángulo.
    Retorna listas de x e y."""
    x, y = [0], [0]
    angle = np.random.uniform(0, 2 * np.pi)
    for _ in range(n_steps):
        angle += np.random.normal(scale=c)
        x.append(x[-1] + np.cos(angle))
        y.append(y[-1] + np.sin(angle))
    return x, y

In [ ]:
def compute_stats(x, y, x0, y0):
    r"""Calcula MDR y ML global."""
    x_arr = np.array(x)
    y_arr = np.array(y)
    distances = np.sqrt((x_arr - x0)**2 + (y_arr - y0)**2)
    mdr = np.mean(distances)
    ml = np.max(distances)
    return mdr, ml

def compute_running_stats(x, y, x0, y0):
    r"""Calcula MDR y ML acumulados para cada paso."""
    x_arr = np.array(x)
    y_arr = np.array(y)
    running_mdr = []
    running_ml = []
    for i in range(1, len(x_arr)+1):
        distances = np.sqrt((x_arr[:i] - x0)**2 + (y_arr[:i] - y0)**2)
        running_mdr.append(np.mean(distances))
        running_ml.append(np.max(distances))
    return list(range(1, len(x_arr)+1)), running_mdr, running_ml

In [ ]:
def plot_brownian_sep(n_steps: int, x0: float, y0: float, stat_choice: str = "MDR"):
    x, y = brownian_motion(n_steps, x0, y0)
    mdr, ml = compute_stats(x, y, x0, y0)
    steps, running_mdr, running_ml = compute_running_stats(x, y, x0, y0)
    stat_values = running_mdr if stat_choice == "MDR" else running_ml
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=list(range(len(x))),
        mode='lines',
        line=dict(color='royalblue', width=4),
        name='Trayectoria'
    ), row=1, col=1)
    fig.add_trace(go.Scatter3d(
        x=[x[-1]], y=[y[-1]], z=[len(x)-1],
        mode='markers+text',
        marker=dict(size=6, color='red'),
        text=[f'MDR: {mdr:.2f}, ML: {ml:.2f}'], textposition='top center',
        name='Estadísticas'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=steps, y=stat_values, mode='lines+markers',
        line=dict(color='green'), name=stat_choice
    ), row=1, col=2)
    fig.update_xaxes(title_text='Paso', row=1, col=2)
    fig.update_yaxes(title_text=stat_choice, row=1, col=2)
    fig.update_layout(title_text=f'Brownian Motion: Trayectoria 3D y {stat_choice} vs Paso',
                      title_x=0.5, margin=dict(l=0, r=0, b=0, t=50))
    return fig

def plot_levy_sep(n_steps: int, alpha: float, loc: float, stat_choice: str = "MDR"):
    x, y = levy_flight(n_steps, alpha, loc)
    mdr, ml = compute_stats(x, y, 0, 0)
    steps, running_mdr, running_ml = compute_running_stats(x, y, 0, 0)
    stat_values = running_mdr if stat_choice == "MDR" else running_ml
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=list(range(len(x))),
        mode='lines',
        line=dict(color='royalblue', width=4),
        name='Trayectoria'
    ), row=1, col=1)
    fig.add_trace(go.Scatter3d(
        x=[x[-1]], y=[y[-1]], z=[len(x)-1],
        mode='markers+text',
        marker=dict(size=6, color='red'),
        text=[f'MDR: {mdr:.2f}, ML: {ml:.2f}'], textposition='top center',
        name='Estadísticas'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=steps, y=stat_values, mode='lines+markers',
        line=dict(color='green'), name=stat_choice
    ), row=1, col=2)
    fig.update_xaxes(title_text='Paso', row=1, col=2)
    fig.update_yaxes(title_text=stat_choice, row=1, col=2)
    fig.update_layout(title_text=f'Lévy Flight: Trayectoria 3D y {stat_choice} vs Paso',
                      title_x=0.5, margin=dict(l=0, r=0, b=0, t=50))
    return fig

def plot_crw_sep(n_steps: int, c: float, stat_choice: str = "MDR"):
    x, y = correlated_random_walk(n_steps, c)
    mdr, ml = compute_stats(x, y, 0, 0)
    steps, running_mdr, running_ml = compute_running_stats(x, y, 0, 0)
    stat_values = running_mdr if stat_choice == "MDR" else running_ml
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=list(range(len(x))),
        mode='lines',
        line=dict(color='royalblue', width=4),
        name='Trayectoria'
    ), row=1, col=1)
    fig.add_trace(go.Scatter3d(
        x=[x[-1]], y=[y[-1]], z=[len(x)-1],
        mode='markers+text',
        marker=dict(size=6, color='red'),
        text=[f'MDR: {mdr:.2f}, ML: {ml:.2f}'], textposition='top center',
        name='Estadísticas'
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=steps, y=stat_values, mode='lines+markers',
        line=dict(color='green'), name=stat_choice
    ), row=1, col=2)
    fig.update_xaxes(title_text='Paso', row=1, col=2)
    fig.update_yaxes(title_text=stat_choice, row=1, col=2)
    fig.update_layout(title_text=f'Correlated Random Walk: Trayectoria 3D y {stat_choice} vs Paso',
                      title_x=0.5, margin=dict(l=0, r=0, b=0, t=50))
    return fig

# Funciones nuevas para cumplir actividades adicionales

def plot_levy_multi_sep(n_steps: int, alpha: float, loc: float, n_curves: int, stat_choice: str = "MDR"):
    """Simula N trayectorias de Lévy Flight y las grafica juntas."""
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5,0.5],
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    colors = ["royalblue", "green", "red", "orange", "purple", "brown", "magenta", "cyan"]
    for i in range(n_curves):
        x, y = levy_flight(n_steps, alpha, loc)
        mdr, ml = compute_stats(x, y, 0, 0)
        steps, running_mdr, running_ml = compute_running_stats(x, y, 0, 0)
        stat_values = running_mdr if stat_choice == "MDR" else running_ml
        color = colors[i % len(colors)]
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=list(range(len(x))),
            mode='lines',
            line=dict(color=color, width=4),
            name=f'Trayectoria {i+1}'
        ), row=1, col=1)
        fig.add_trace(go.Scatter3d(
            x=[x[-1]], y=[y[-1]], z=[len(x)-1],
            mode='markers+text',
            marker=dict(size=6, color=color),
            text=[f'MDR: {mdr:.2f}, ML: {ml:.2f}'], textposition='top center',
            name=f'Estadísticas {i+1}'
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=steps, y=stat_values, mode='lines+markers',
            line=dict(color=color),
            name=f'{stat_choice} {i+1}'
        ), row=1, col=2)
    fig.update_xaxes(title_text='Paso', row=1, col=2)
    fig.update_yaxes(title_text=stat_choice, row=1, col=2)
    fig.update_layout(title_text=f'Lévy Flight (Multi): {n_curves} Trayectorias y {stat_choice} vs Paso',
                      title_x=0.5, margin=dict(l=0, r=0, b=0, t=50))
    return fig

def plot_levy_hist(n_samples: int, alpha: float, loc: float):
    """Genera un histograma de muestras de la distribución Lévy y superpone la PDF teórica."""
    samples = np.random.pareto(alpha, n_samples) + loc
    hist = go.Histogram(x=samples, histnorm='density', name='Histograma', opacity=0.75)
    x_min = loc
    x_max = np.percentile(samples, 99)
    x_values = np.linspace(x_min, x_max, 100)
    pdf_values = alpha / (1 + x_values - loc)**(alpha+1)
    pdf_curve = go.Scatter(x=x_values, y=pdf_values, mode='lines', name='PDF Teórica', line=dict(color='red'))
    fig = go.Figure(data=[hist, pdf_curve])
    fig.update_layout(title_text=f'Lévy Distribution: Histograma y PDF (alpha={alpha}, loc={loc})',
                      xaxis_title='Valor',
                      yaxis_title='Densidad')
    return fig

def plot_crw_multi(n_steps: int, c: float, n_traj: int, stat_choice: str = "MDR"):
    """Simula N trayectorias de Correlated Random Walk y las grafica juntas."""
    fig = make_subplots(rows=1, cols=2, column_widths=[0.5,0.5],
                        specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    colors = ["royalblue", "green", "red", "orange", "purple", "brown", "magenta", "cyan"]
    for i in range(n_traj):
        x, y = correlated_random_walk(n_steps, c)
        mdr, ml = compute_stats(x, y, 0, 0)
        steps, running_mdr, running_ml = compute_running_stats(x, y, 0, 0)
        stat_values = running_mdr if stat_choice == "MDR" else running_ml
        color = colors[i % len(colors)]
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=list(range(len(x))),
            mode='lines',
            line=dict(color=color, width=4),
            name=f'Trayectoria {i+1}'
        ), row=1, col=1)
        fig.add_trace(go.Scatter3d(
            x=[x[-1]], y=[y[-1]], z=[len(x)-1],
            mode='markers+text',
            marker=dict(size=6, color=color),
            text=[f'MDR: {mdr:.2f}, ML: {ml:.2f}'], textposition='top center',
            name=f'Estadísticas {i+1}'
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=steps, y=stat_values, mode='lines+markers',
            line=dict(color=color),
            name=f'{stat_choice} {i+1}'
        ), row=1, col=2)
    fig.update_xaxes(title_text='Paso', row=1, col=2)
    fig.update_yaxes(title_text=stat_choice, row=1, col=2)
    fig.update_layout(title_text=f'Correlated Random Walk (Multi): {n_traj} Trayectorias y {stat_choice} vs Paso',
                      title_x=0.5, margin=dict(l=0, r=0, b=0, t=50))
    return fig

In [ ]:
# Contenedor global para los gráficos
plot_output = widgets.Output()

# Contenedor para los controles
controls_container = widgets.VBox()

# Dropdown global con las opciones extendidas
walk_type_widget = widgets.Dropdown(options=[
    'Brownian Motion',
    'Lévy Flight (Single)',
    'Lévy Flight (Multi)',
    'Lévy Histogram',
    'Correlated Random Walk (Single)',
    'Correlated Random Walk (Multi)'
],
                                     value='Brownian Motion',
                                     description='Tipo de caminata:')

# Controles para cada opción

# Brownian Motion
brownian_controls = {
    'n_steps': widgets.IntSlider(min=50, max=1000, step=50, value=200, description='Pasos:'),
    'x0': widgets.FloatSlider(min=-10, max=10, step=1, value=0, description='X Inicial:'),
    'y0': widgets.FloatSlider(min=-10, max=10, step=1, value=0, description='Y Inicial:'),
    'stat_choice': widgets.Dropdown(options=['MDR', 'ML'], value='MDR', description='Estadística:')
}

# Lévy Flight (Single)
levy_single_controls = {
    'n_steps': widgets.IntSlider(min=50, max=1000, step=50, value=200, description='Pasos:'),
    'alpha': widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.5, description='Alpha:'),
    'loc': widgets.FloatSlider(min=0, max=10, step=0.5, value=1, description='Localización:'),
    'stat_choice': widgets.Dropdown(options=['MDR', 'ML'], value='MDR', description='Estadística:')
}

# Lévy Flight (Multi)
levy_multi_controls = {
    'n_steps': widgets.IntSlider(min=50, max=1000, step=50, value=200, description='Pasos:'),
    'alpha': widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.5, description='Alpha:'),
    'loc': widgets.FloatSlider(min=0, max=10, step=0.5, value=1, description='Localización:'),
    'n_curves': widgets.IntSlider(min=2, max=10, step=1, value=3, description='Curvas:'),
    'stat_choice': widgets.Dropdown(options=['MDR', 'ML'], value='MDR', description='Estadística:')
}

# Lévy Histogram
levy_hist_controls = {
    'n_samples': widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='Muestras:'),
    'alpha': widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.5, description='Alpha:'),
    'loc': widgets.FloatSlider(min=0, max=10, step=0.5, value=1, description='Localización:')
}

# Correlated Random Walk (Single)
crw_single_controls = {
    'n_steps': widgets.IntSlider(min=50, max=1000, step=50, value=200, description='Pasos:'),
    'c': widgets.FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='C:'),
    'stat_choice': widgets.Dropdown(options=['MDR', 'ML'], value='MDR', description='Estadística:')
}

# Correlated Random Walk (Multi)
crw_multi_controls = {
    'n_steps': widgets.IntSlider(min=50, max=1000, step=50, value=200, description='Pasos:'),
    'c': widgets.FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='C:'),
    'n_traj': widgets.IntSlider(min=2, max=10, step=1, value=3, description='Trayectorias:'),
    'stat_choice': widgets.Dropdown(options=['MDR', 'ML'], value='MDR', description='Estadística:')
}

# Función para actualizar el gráfico según la opción seleccionada
def update_plot(movement_type):
    with plot_output:
        plot_output.clear_output()
        if movement_type == 'Brownian Motion':
            fig = plot_brownian_sep(brownian_controls['n_steps'].value,
                                    brownian_controls['x0'].value,
                                    brownian_controls['y0'].value,
                                    brownian_controls['stat_choice'].value)
        elif movement_type == 'Lévy Flight (Single)':
            fig = plot_levy_sep(levy_single_controls['n_steps'].value,
                                levy_single_controls['alpha'].value,
                                levy_single_controls['loc'].value,
                                levy_single_controls['stat_choice'].value)
        elif movement_type == 'Lévy Flight (Multi)':
            fig = plot_levy_multi_sep(levy_multi_controls['n_steps'].value,
                                      levy_multi_controls['alpha'].value,
                                      levy_multi_controls['loc'].value,
                                      levy_multi_controls['n_curves'].value,
                                      levy_multi_controls['stat_choice'].value)
        elif movement_type == 'Lévy Histogram':
            fig = plot_levy_hist(levy_hist_controls['n_samples'].value,
                                 levy_hist_controls['alpha'].value,
                                 levy_hist_controls['loc'].value)
        elif movement_type == 'Correlated Random Walk (Single)':
            fig = plot_crw_sep(crw_single_controls['n_steps'].value,
                               crw_single_controls['c'].value,
                               crw_single_controls['stat_choice'].value)
        elif movement_type == 'Correlated Random Walk (Multi)':
            fig = plot_crw_multi(crw_multi_controls['n_steps'].value,
                                 crw_multi_controls['c'].value,
                                 crw_multi_controls['n_traj'].value,
                                 crw_multi_controls['stat_choice'].value)
        fig.show()

# Función para actualizar el dashboard (controles y gráfico) al cambiar la opción
def update_dashboard(change=None):
    if walk_type_widget.value == 'Brownian Motion':
        controls_container.children = list(brownian_controls.values())
        for ctrl in brownian_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Brownian Motion'), names='value')
        update_plot('Brownian Motion')
    elif walk_type_widget.value == 'Lévy Flight (Single)':
        controls_container.children = list(levy_single_controls.values())
        for ctrl in levy_single_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Lévy Flight (Single)'), names='value')
        update_plot('Lévy Flight (Single)')
    elif walk_type_widget.value == 'Lévy Flight (Multi)':
        controls_container.children = list(levy_multi_controls.values())
        for ctrl in levy_multi_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Lévy Flight (Multi)'), names='value')
        update_plot('Lévy Flight (Multi)')
    elif walk_type_widget.value == 'Lévy Histogram':
        controls_container.children = list(levy_hist_controls.values())
        for ctrl in levy_hist_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Lévy Histogram'), names='value')
        update_plot('Lévy Histogram')
    elif walk_type_widget.value == 'Correlated Random Walk (Single)':
        controls_container.children = list(crw_single_controls.values())
        for ctrl in crw_single_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Correlated Random Walk (Single)'), names='value')
        update_plot('Correlated Random Walk (Single)')
    elif walk_type_widget.value == 'Correlated Random Walk (Multi)':
        controls_container.children = list(crw_multi_controls.values())
        for ctrl in crw_multi_controls.values():
            ctrl.unobserve_all()
            ctrl.observe(lambda change: update_plot('Correlated Random Walk (Multi)'), names='value')
        update_plot('Correlated Random Walk (Multi)')

    dashboard.children = [walk_type_widget, controls_container, plot_output]

# Observador para el dropdown global
walk_type_widget.observe(lambda change: update_dashboard(), names='value')

# Configurar y mostrar el dashboard inicial
dashboard = widgets.VBox([walk_type_widget, controls_container, plot_output])
update_dashboard()
display(dashboard)